In [1]:
% setup MRI-education-resources path and requirements
cd ../
startup

loading image
loading signal


# Image Reconstruction (In Progress Page)

Image reconstruction in MRI means converting the raw signal collected in k-space into images.  For a fully-sampled acquisition using Cartesian sampling (e.g. on a grid), this can be done with a Fourier Transform.  However, the majority of MRI scans are now under-sampled, meaning some samples are skipped to accelerate the scan, which requires more complex image reconstruction methods.


## Formulating the Reconstruction Problem

For image reconstruction, it is helpful to reformulate the MRI Signal Equation, which we can pose for MRI as a linear system and using standard mathematical notation describing systems as:

$$ \mathbf{y} = \mathbf{Ex} + \mathbf{n} $$

where $\mathbf{y}$ is the acquired data, $\mathbf{E}$ is the encoding matrix, $\mathbf{x}$ is the spatial distribution of the transverse magnetization (e.g. image), and $\mathbf{n}$ is noise

The encoding matrix, $E$, must include a discrete Fourier Transform matrix, representing our data is the Fourier Transform of the transverse magnetization, evaluated at k-space locations.  It should also include coil sensitivity profiles in order to support parallel imaging formulations.

## Partial Fourier

How does it work?

How is it implemented (k-space sampling, reconstruction algorithm)?

When can it be used?

## Parallel Imaging

For parallel imaging, we need to consider the coil sensitivity profiles, $\mathbf{C}$, into encoding matrix along with a Fourier Transform encoding matrix, $\mathbf{F}$:

$$\mathbf{E} = \mathbf{F} \mathbf{C}$$

### Image-space Methods
Image-space parallel imaging (e.g. SENSE) can be formulated as the following optimization problem

$$\hat{x}_{PI} = \arg \min_\mathbf{x} \frac{1}{2} \| \mathbf{y} - \mathbf{Ex} \|^2_2 = (\mathbf{E}^H\mathbf{E})^{-1} \mathbf{E}^H \mathbf{y} $$

In other words, parallel can be solved directly by a pseudo-inverse, but requires estimates of the coil sensitivity profiles.

### K-space Methods

K-space parallel imaging (e.g. GRAPPA) utilize a calibration kernel, computed from the data itself and captured in the matrix $\mathbf{G}$.  These can also be generally formulated as the following optimization problem

$$\hat{x}_{PI} = \arg \min_\mathbf{x} \| \mathbf{y} - \mathbf{Ex} \|^2_2 + \lambda \| (\mathbf{G} - \mathbf{I}) \mathbf{x} \|^2_2 $$

Where $\lambda$ is a tuning parameter and $\mathbf{I}$ is the identity matrix.  This simultaneous enforces data consistency (first term) as well as self-consistency of the calibration (sescond term).



## Compressed Sensing Methods

Compressed Sensing theory says that an image that is compressible in some domain can then be reconstructed from a subset of data samples.  In other words, we can further accelerate our data acquisition.

Compressed Sensing is formulated as the following optimization problem

$$\hat{x}_{CS} = \arg \min_\mathbf{x} \frac{1}{2} \| \mathbf{y} = \mathbf{Ex} \|^2_2 +\tau \| \mathbf{Wx} \|_1 $$

which includes a data consistency term where the data multiplied by the encoding matrix must match the reconstructed image, and a regularization term that enforces that the image is sparse in some other domain through the sparsifying transform, $\mathbf{W}$.

## Neural Network Reconstructions